In [166]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf
import scipy
import math
import pandas_ta as ta
from pathlib import Path
import yfinance as yf
from datetime import datetime, timedelta

In [167]:
nas_path = Path('./nasdaq_2019_to_present.csv')
nas_df = pd.read_csv(nas_path)
nas_df['date'] = pd.to_datetime(nas_df['date'])
nas_df = nas_df.set_index('date')
nas_df.columns = ['close', 'open', 'high', 'low']
nas_df

,close,open,high,low
date,,,,
2024-01-21,17462.75,17462.75,17462.75,17462.75
2024-01-20,17462.75,17462.75,17462.75,17462.75
2024-01-19,17438.50,17116.00,17471.25,17107.25
2024-01-18,17110.00,16854.50,17125.75,16834.25
2024-01-17,16869.75,16978.00,16982.00,16689.25
...,...,...,...,...
2020-07-07,10532.25,10600.00,10694.50,10505.75
2020-07-06,10598.50,10347.25,10614.50,10341.50
2020-07-03,10328.88,10348.00,10389.25,10310.20


In [168]:
end_date = datetime.now()
start_date = end_date - timedelta(days = 1825) # 60 Days is for 5 min data
start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')
#start = start_date_str, end = end_date_str

In [169]:
security_df = yf.Ticker('NQ=F')
security_df.info
hist = security_df.history(start = start_date_str, end =  end_date_str, interval = '1D') # start = '2023-11-23', end = '2024-01-21'
                                                                 #Valid intervals: [1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo]
nasdaq_df = pd.DataFrame(hist[['Volume', 'Close', 'High', 'Low']])
df_cumulative_delta = pd.DataFrame(hist[['Volume', 'Close', 'High', 'Low']])
nasdaq_df


,Volume,Close,High,Low
Date,,,,
2019-01-22 00:00:00-05:00,651258,6653.00,6794.50,6611.50
2019-01-23 00:00:00-05:00,515044,6665.25,6724.25,6592.25
2019-01-24 00:00:00-05:00,449017,6675.00,6718.50,6653.00
2019-01-25 00:00:00-05:00,446822,6791.25,6810.50,6676.75
2019-01-28 00:00:00-05:00,490045,6701.75,6791.00,6648.25
...,...,...,...,...
2024-01-15 00:00:00-05:00,593445,16970.00,17024.25,16936.25
2024-01-16 00:00:00-05:00,816021,16966.50,17033.75,16811.75
2024-01-17 00:00:00-05:00,722784,16869.75,16982.00,16689.25


In [170]:
def find_levels( 
        price: np.array, atr: float, # Log closing price, and log atr 
        first_w: float = 0.1, 
        atr_mult: float = 3.0, 
        prom_thresh: float = 0.1
):

    # Setup weights
    last_w = 1.0
    w_step = (last_w - first_w) / len(price)
    weights = first_w + np.arange(len(price)) * w_step
    weights[weights < 0] = 0.0

    # Get kernel of price. 
    kernal = scipy.stats.gaussian_kde(price, bw_method=atr*atr_mult, weights=weights)

    # Construct market profile
    min_v = np.min(price)
    max_v = np.max(price)
    step = (max_v - min_v) / 200
    price_range = np.arange(min_v, max_v, step)
    pdf = kernal(price_range) # Market profile

    # Find significant peaks in the market profile
    pdf_max = np.max(pdf)
    prom_min = pdf_max * prom_thresh

    peaks, props = scipy.signal.find_peaks(pdf, prominence=prom_min)
    levels = [] 
    for peak in peaks:
        levels.append(np.exp(price_range[peak]))

    return levels, peaks, props, price_range, pdf, weights


def support_resistance_levels(
        data, lookback: int, 
        first_w: float = 0.03, atr_mult: float = 3.5, prom_thresh: float = 0.20
):
    # Check if necessary columns are present in the DataFrame
    if 'high' not in data.columns or 'low' not in data.columns or 'close' not in data.columns:
        raise KeyError("Columns 'high', 'low', and 'close' must be present in the DataFrame.")

    # Get log average true range
    atr = ta.atr(np.log(data['high']), np.log(data['low']), np.log(data['close']), lookback)

    all_levels = [None] * len(data)
    for i in range(lookback, len(data)):
        i_start = i - lookback
        vals = np.log(data.iloc[i_start + 1: i + 1]['close'].to_numpy())
        levels, peaks, props, price_range, pdf, weights = find_levels(vals, atr.iloc[i], first_w, atr_mult, prom_thresh)
        all_levels[i] = levels

    return all_levels


def sr_penetration_signal(data, levels: list):
    signal = np.zeros(len(data))
    curr_sig = 0.0
    close_arr = data['close'].to_numpy()
    for i in range(1, len(data)):
        if levels[i] is None:
            continue

        last_c = close_arr[i - 1]
        curr_c = close_arr[i]

        
        for level in levels[i]:
            if curr_c > level and last_c <= level: # Close cross above line
                curr_sig = 1.0
            elif curr_c < level and last_c >= level: # Close cross below line
                curr_sig = -1.0

        signal[i] = curr_sig
    return signal

def get_trades_from_signal(data, signal: np.array):
    long_trades = []
    short_trades = []

    close_arr = data['close'].to_numpy()
    last_sig = 0.0
    open_trade = None
    idx = data.index
    for i in range(len(data)):
        if signal[i] == 1.0 and last_sig != 1.0: # Long entry
            if open_trade is not None:
                open_trade[2] = idx[i] 
                open_trade[3] = close_arr[i]
                short_trades.append(open_trade)

            open_trade = [idx[i], close_arr[i], -1, np.nan]
        if signal[i] == -1.0  and last_sig != -1.0: # Short entry
            if open_trade is not None:
                open_trade[2] = idx[i] 
                open_trade[3] = close_arr[i]
                long_trades.append(open_trade)

            open_trade = [idx[i], close_arr[i], -1, np.nan]

        last_sig = signal[i]

    long_trades = pd.DataFrame(long_trades, columns=['entry_time', 'entry_price', 'exit_time', 'exit_price'])
    short_trades = pd.DataFrame(short_trades, columns=['entry_time', 'entry_price', 'exit_time', 'exit_price'])

    long_trades['percent'] = (long_trades['exit_price'] - long_trades['entry_price']) / long_trades['entry_price'] 
    short_trades['percent'] = -1 * (short_trades['exit_price'] - short_trades['entry_price']) / short_trades['entry_price']
    long_trades = long_trades.set_index('entry_time')
    short_trades = short_trades.set_index('entry_time')
    return long_trades, short_trades 




In [171]:

if __name__ == '__main__':
   
    # Trend following strategy
    data = nas_df
    plt.style.use('dark_background') 
    levels = support_resistance_levels(data, 365, first_w=1.0, atr_mult=3.0)

    data['sr_signal'] = sr_penetration_signal(data, levels)
    data['log_ret'] = np.log(data['close']).diff().shift(-1)
    data['sr_return'] = data['sr_signal'] * data['log_ret']

    long_trades, short_trades = get_trades_from_signal(data, data['sr_signal'].to_numpy())

    

In [173]:
#Calcualting HLC3 value
nasdaq_df['HLC3'] = (nasdaq_df['High'] + nasdaq_df['Low'] + nasdaq_df['Close']) / 3
# Calculating VWAP (using HLC3)
nasdaq_df['VWAP'] = (nasdaq_df['HLC3'] * nasdaq_df['Volume']).cumsum() / nasdaq_df['Volume'].cumsum()
#Calculating Standard Deviation
nasdaq_df['PriceDeviation'] = (nasdaq_df['HLC3'] - nasdaq_df['VWAP']).rolling(window = 78).std() ##Window 78 set based on 5 min trading data (78 5 min in 1 day)##
# Session based (How it was on Pinescript) appears to be based on daily
# Calculate bands [UpperBand_1, LowerBand_1, UpperBand_2, LowerBand_2, UpperBand_3, LowerBand_3]
for i in [1, 2, 3]:
    nasdaq_df[f'UpperBand_{i}'] = nasdaq_df['VWAP'] + (i * nasdaq_df['PriceDeviation'])
    nasdaq_df[f'LowerBand_{i}'] = nasdaq_df['VWAP'] - (i * nasdaq_df['PriceDeviation'])
nasdaq_df = nasdaq_df.dropna()
df_hlc3_vwap = nasdaq_df
df_hlc3_vwap

,Volume,Close,High,Low,HLC3,VWAP,PriceDeviation,UpperBand_1,LowerBand_1,UpperBand_2,LowerBand_2,UpperBand_3,LowerBand_3
Date,,,,,,,,,,,,,
2019-05-13 00:00:00-04:00,773473,7314.75,7551.50,7305.00,7390.416667,7312.836158,176.944687,7489.780845,7135.891472,7666.725532,6958.946785,7843.670219,6782.002098
2019-05-14 00:00:00-04:00,547769,7421.00,7459.50,7290.00,7390.166667,7314.040518,175.458428,7489.498947,7138.582090,7664.957375,6963.123661,7840.415804,6787.665233
2019-05-15 00:00:00-04:00,535915,7529.25,7538.50,7353.75,7473.833333,7316.438764,172.470163,7488.908927,7143.968601,7661.379091,6971.498438,7833.849254,6799.028275
2019-05-16 00:00:00-04:00,525588,7600.25,7641.00,7473.50,7571.583333,7320.139827,169.120542,7489.260369,7151.019285,7658.380911,6981.898743,7827.501453,6812.778201
2019-05-17 00:00:00-04:00,626777,7511.50,7620.00,7506.50,7546.000000,7323.980425,167.096717,7491.077142,7156.883708,7658.173859,6989.786991,7825.270577,6822.690274
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-15 00:00:00-05:00,593445,16970.00,17024.25,16936.25,16976.833333,12182.785209,767.416123,12950.201332,11415.369087,13717.617455,10647.952964,14485.033577,9880.536842
2024-01-16 00:00:00-05:00,816021,16966.50,17033.75,16811.75,16937.333333,12188.392927,769.918357,12958.311284,11418.474570,13728.229641,10648.556213,14498.147998,9878.637856
2024-01-17 00:00:00-05:00,722784,16869.75,16982.00,16689.25,16847.000000,12193.254610,769.109108,12962.363718,11424.145502,13731.472827,10655.036394,14500.581935,9885.927286


In [ ]:
long_trades['total_points'] = long_trades['exit_price'] - long_trades['entry_price']
long_trades['pnl_1_contract'] = long_trades['total_points'] * 20
long_trades

,entry_price,exit_time,exit_price,percent,total_points,pnl_1_contract
entry_time,,,,,,
2022-09-21,11710.00,2022-08-09,13031.50,0.112852,1321.50,26430.0
2022-08-05,13228.75,2022-08-02,12924.50,-0.022999,-304.25,-6085.0
2022-07-08,12152.00,2022-07-06,11880.25,-0.022363,-271.75,-5435.0
2022-06-29,11691.00,2022-06-22,11565.75,-0.010713,-125.25,-2505.0
2022-06-10,11840.00,2022-03-28,14985.25,0.265646,3145.25,62905.0
2021-11-08,16327.75,2021-11-03,16129.75,-0.012127,-198.00,-3960.0
2021-10-01,14761.75,2021-09-30,14682.50,-0.005369,-79.25,-1585.0
2021-09-29,14739.75,2021-08-31,15582.50,0.057175,842.75,16855.0
2021-08-30,15597.50,2021-08-28,15428.25,-0.010851,-169.25,-3385.0


In [ ]:
long_pnl = long_trades['pnl_1_contract'].sum()
long_pnl

60590.0

In [ ]:
short_trades['total_points'] = short_trades['entry_price'] - short_trades['exit_price']
short_trades['pnl_1_contract'] = short_trades['total_points'] * 20
short_trades

,entry_price,exit_time,exit_price,percent,total_points,pnl_1_contract
entry_time,,,,,,
2022-08-09,13031.50,2022-08-05,13228.75,-0.015136,-197.25,-3945.0
2022-08-02,12924.50,2022-07-08,12152.00,0.059770,772.50,15450.0
2022-07-06,11880.25,2022-06-29,11691.00,0.015930,189.25,3785.0
2022-06-22,11565.75,2022-06-10,11840.00,-0.023712,-274.25,-5485.0
2022-03-28,14985.25,2021-11-08,16327.75,-0.089588,-1342.50,-26850.0
2021-11-03,16129.75,2021-10-01,14761.75,0.084812,1368.00,27360.0
2021-09-30,14682.50,2021-09-29,14739.75,-0.003899,-57.25,-1145.0
2021-08-31,15582.50,2021-08-30,15597.50,-0.000963,-15.00,-300.0
2021-08-28,15428.25,2021-07-29,15037.75,0.025311,390.50,7810.0


In [ ]:
short_pnl = short_trades['pnl_1_contract'].sum()
short_pnl

72955.0